In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score
import os
import pandas as pd
import numpy as np
from sklearn.utils import check_random_state
from tqdm import tqdm
import matplotlib.pyplot as plt
import glob

# load data

In [16]:
path = os.getcwd()
par = os.path.abspath(os.path.join(path, os.pardir))

data_path = os.path.join(par,'3_generate_features','dimensionless_cropped_final_feature_array.csv')
label_path = os.path.join(par,'3_generate_features','final_label_array.csv')

os.path.isfile(data_path)

feat_df = pd.read_csv(data_path, index_col=0)
label_df = pd.read_csv(label_path)

# define functions

In [17]:
#Function to change multiclass classification to 1 vs all
def multiclass_to_binary(labels, most_common_id):
    to_binary = lambda val: 1 if val == most_common_id else 0
    to_binary_vec = np.vectorize(to_binary)

    labels_1vsall = to_binary_vec(labels)

    return labels_1vsall


def subsample_data(features,labels_1vsall,n_subsample):

    rng = np.random.default_rng()

    #Dominant class boolean index
    positive_class_mask = labels_1vsall==1
    
    #Dominant class indexing to grab for training/test set (we want 50/50 representation)
    SR_features_train = features[positive_class_mask,:]
    SR_target_train = labels_1vsall[positive_class_mask]

    #Grabbing all negative examples of which we're going to grab a number equal to the number of dominant class
    SR_negative_train = features[~positive_class_mask,:]
    SR_negative_target = labels_1vsall[~positive_class_mask]

    #Apply subsampling. We grab a random subset from the negative set of the same size as the positive examples
    subsample_idx = rng.permutation(SR_negative_target.size)[:n_subsample]

    #Concatenate an equal amount of negative training data to the list of positive training data so we have 50/50 class representation
    SR_features = np.concatenate((SR_features_train, SR_negative_train[subsample_idx,:]),axis=0)
    SR_targets = np.concatenate((SR_target_train, SR_negative_target[subsample_idx]), axis=0)

    return SR_features, SR_targets


def run_random_forest(features,labels_1vsall):
    #Stratified K fold (maintain class balance)

    skf = StratifiedKFold(n_splits=10)

    cv_precisions = []
    cv_recalls = []

    aggregated_feature_importances = []
    
    for i, (train_idx, test_idx) in enumerate(skf.split(features,labels_1vsall)):
        train_features_i = features[train_idx]
        train_labels_i = labels_1vsall[train_idx]

        test_features_i = features[test_idx]
        test_labels_i = labels_1vsall[test_idx]

        #Fit naive rf model
        naive_rf_i = RandomForestClassifier()
        naive_rf_i.fit(train_features_i, train_labels_i)
        
        predict_labels_i = naive_rf_i.predict(test_features_i)

        #Fit on k-folded validation set
        precision, recall = precision_score(test_labels_i, predict_labels_i), recall_score(test_labels_i, predict_labels_i)

        aggregated_feature_importances += [np.array(naive_rf_i.feature_importances_)]
        
        cv_precisions += [precision]
        cv_recalls += [recall]

    aggregated_feature_importances = np.array(aggregated_feature_importances)
    aggregated_feature_importances = np.mean(aggregated_feature_importances, axis=0)
    sorted_indices = np.argsort(aggregated_feature_importances)
    sorted_importances = aggregated_feature_importances[sorted_indices]
    sorted_feature_names = np.array(feature_names)[sorted_indices]
    feature_info = pd.DataFrame(data={'Name':list(sorted_feature_names),
                                      
    'Importances':sorted_importances}).sort_values(by='Importances',ascending=False,inplace=False)
    feature_info.to_csv(f'features/{feature_list_number}_iteration_feature_{len(feature_names)}_features.csv')
    return np.mean(cv_precisions), np.mean(cv_recalls)

# prune correlated features


In [18]:
corr = feat_df.corr().abs()

In [19]:
feat_df

,e1_avg_oxi_pos/e1_avg_oxi_neg,(e1_avg_oxi_pos+e1_avg_oxi_neg)/e1_avg_oxi_pos,(e1_avg_oxi_pos-e1_avg_oxi_neg)/e1_avg_oxi_pos,(e1_avg_oxi_pos+e1_avg_oxi_neg)/e1_avg_oxi_neg,(e1_avg_oxi_pos-e1_avg_oxi_neg)/e1_avg_oxi_neg,e1_avg_oxi_pos/e2_avg_oxi_pos,(e1_avg_oxi_pos+e2_avg_oxi_pos)/e1_avg_oxi_pos,(e1_avg_oxi_pos-e2_avg_oxi_pos)/e1_avg_oxi_pos,(e1_avg_oxi_pos+e2_avg_oxi_pos)/e2_avg_oxi_pos,(e1_avg_oxi_pos-e2_avg_oxi_pos)/e2_avg_oxi_pos,...,(e1_Electronegativity+e2_Electronegativity)/e2_Electronegativity,(e1_Electronegativity-e2_Electronegativity)/e2_Electronegativity,e1_GSvolume_pa/e2_GSvolume_pa,(e1_GSvolume_pa+e2_GSvolume_pa)/e1_GSvolume_pa,(e1_GSvolume_pa-e2_GSvolume_pa)/e1_GSvolume_pa,(e1_GSvolume_pa+e2_GSvolume_pa)/e2_GSvolume_pa,(e1_GSvolume_pa-e2_GSvolume_pa)/e2_GSvolume_pa,max_ionic_char,avg_ionic_char,chg_dispro
0,5.0,1.2,0.8,6.0,4.0,5.0,1.2,0.8,6.0,4.0,...,1.447674,-0.552326,1.833059,1.545536,0.454464,2.833059,0.833059,0.594445,0.132099,0
1,5.0,1.2,0.8,6.0,4.0,5.0,1.2,0.8,6.0,4.0,...,1.552326,-0.447674,2.244920,1.445450,0.554550,3.244920,1.244920,0.447278,0.099395,0
2,5.0,1.2,0.8,6.0,4.0,5.0,1.2,0.8,6.0,4.0,...,1.450581,-0.549419,1.151849,1.868169,0.131831,2.151849,0.151849,0.590585,0.131241,0
3,1.0,2.0,0.0,2.0,0.0,1.0,2.0,0.0,2.0,0.0,...,1.793939,-0.206061,1.639685,1.609873,0.390127,2.639685,0.639685,0.028486,0.006330,0
4,5.0,1.2,0.8,6.0,4.0,5.0,1.2,0.8,6.0,4.0,...,1.552326,-0.447674,2.244920,1.445450,0.554550,3.244920,1.244920,0.447278,0.099395,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8743,1.0,2.0,0.0,2.0,0.0,1.0,2.0,0.0,2.0,0.0,...,3.062176,1.062176,0.594458,2.682204,-0.682204,1.594458,-0.405542,0.650281,0.144507,0
8744,1.0,2.0,0.0,2.0,0.0,1.0,2.0,0.0,2.0,0.0,...,1.595930,-0.404070,3.466227,1.288498,0.711502,4.466227,2.466227,0.383086,0.085130,0
8745,1.0,2.0,0.0,2.0,0.0,1.0,2.0,0.0,2.0,0.0,...,1.601852,-0.398148,1.414914,1.706757,0.293243,2.414914,0.414914,0.168813,0.037514,0
8746,5.0,1.2,0.8,6.0,4.0,5.0,1.2,0.8,6.0,4.0,...,1.627907,-0.372093,1.723229,1.580306,0.419694,2.723229,0.723229,0.336084,0.074685,0


In [20]:
feats_to_prune = set([])
thresh = 0.95
immune = set([])

for feat in feat_df.columns:
    # If we already pruned this feature, we're done here
    if feat in feats_to_prune: continue
    # If not, make it immune and discard things correlated with it
    immune.update({feat})
    correlated_features = set(corr[corr[feat] > thresh].index)
    feats_to_prune.update(correlated_features - immune)
feats_to_prune
feat_df = feat_df.drop(columns=feats_to_prune)

# prune based on RF importance

In [22]:
nfeats, precs, recalls = [], [], []

feature_names = np.zeros(999999) #initializing counter for the while loop. dont judge me

while len(feature_names) > 1:
    print(f'{len(feature_names)} features remaining')

    #Dataframe convert label into categorical variable for classification
    #Then convert labels into numpy array
    label_name = 'Prototype'
    label_df[label_name]= pd.Categorical(label_df[label_name])
    label_df['numeric_label'] = label_df[label_name].cat.codes

    #Convert numerical dataframe column to array
    labels = label_df['numeric_label'].to_numpy()

    #Convert features to numpy
    #Also define feature names for symbolic regression
    features = feat_df.to_numpy()
    feature_names = np.array(feat_df.columns)

    #Define most class id of interest and relabel
#     id_ofinterest = 203 #laves
    id_ofinterest = 151 #fluorite

    labels_1vsall = multiclass_to_binary(labels, most_common_id=id_ofinterest)
    
    # #Set rng seed and permutation of data examples for training
    # rng = check_random_state(5)

    #Check number of dominant class examples
    n_positive_class = np.sum(labels_1vsall)
    
    def feature_list_number_func(filename):
        return int(filename.split('_')[0].split('/')[-1])

    previous_features = glob.glob('features/*')
    previous_features_sorted = sorted(previous_features,key=feature_list_number_func)

    if len(previous_features_sorted) == 0: 
        feature_list_number = 1
        pass
    else:
        previous_feature_data = pd.read_csv(previous_features_sorted[-1]) # read most recent feature reduction iteration
        feature_list_number = feature_list_number_func(previous_features_sorted[-1]) + 1 # get current feature reduction iteration
        previous_feature_keep = previous_feature_data.query(f"Importances > {previous_feature_data['Importances'].quantile(q=0.25)}") # get list of feature names with importances in the top 75%
        previous_feature_names = previous_feature_keep['Name']

        keep = np.isin(feature_names, previous_feature_names) # identify feature names that need to be kept

        feature_names = feature_names[keep]
        
    ### Train, and get feature importances
    subsampled_features, subsampled_targets = subsample_data(feat_df[feature_names].to_numpy(),labels_1vsall,n_positive_class)
    prec, recall = run_random_forest(subsampled_features, subsampled_targets)
    
    ### Record
    nfeats.append(len(feature_names))
    precs.append(prec)
    recalls.append(recall)

999999 features remaining
332 features remaining
249 features remaining
186 features remaining
139 features remaining
104 features remaining
78 features remaining
58 features remaining
43 features remaining
32 features remaining
24 features remaining
18 features remaining
13 features remaining
9 features remaining
6 features remaining
4 features remaining
3 features remaining
2 features remaining


In [23]:
feat_df

,e1_avg_oxi_pos/e1_avg_oxi_neg,(e1_avg_oxi_pos+e1_avg_oxi_neg)/e1_avg_oxi_pos,e1_avg_oxi_neg/e2_avg_oxi_pos,e1_MendeleevNumber/e2_MendeleevNumber,(e1_MendeleevNumber+e2_MendeleevNumber)/e1_MendeleevNumber,e1_Column/e1_Row,(e1_Column+e1_Row)/e1_Column,e1_Column/e2_Column,(e1_Column+e2_Column)/e1_Column,e1_Column/e2_Row,...,e2_NdUnfilled/e2_NUnfilled,(e2_NdUnfilled+e2_NUnfilled)/e2_NdUnfilled,e2_NfUnfilled/e2_NUnfilled,(e2_NfUnfilled+e2_NUnfilled)/e2_NfUnfilled,e1_Electronegativity/e2_Electronegativity,(e1_Electronegativity+e2_Electronegativity)/e1_Electronegativity,e1_GSvolume_pa/e2_GSvolume_pa,(e1_GSvolume_pa+e2_GSvolume_pa)/e1_GSvolume_pa,max_ionic_char,chg_dispro
0,5.0,1.2,1.0,0.494253,3.023256,1.000000,2.000000,0.250000,5.000000,2.0,...,0.333333,4.000000,0.333333,4.0,0.447674,3.233766,1.833059,1.545536,0.594445,0
1,5.0,1.2,1.0,0.896552,2.115385,4.666667,1.214286,0.875000,2.142857,7.0,...,0.333333,4.000000,0.333333,4.0,0.552326,2.810526,2.244920,1.445450,0.447278,0
2,5.0,1.2,1.0,0.597701,2.673077,1.750000,1.571429,0.437500,3.285714,3.5,...,0.333333,4.000000,0.333333,4.0,0.450581,3.219355,1.151849,1.868169,0.590585,0
3,1.0,2.0,1.0,0.985507,2.014706,0.666667,2.500000,0.166667,7.000000,0.5,...,1.000000,2.000000,1.000000,2.0,0.793939,2.259542,1.639685,1.609873,0.028486,0
4,5.0,1.2,1.0,0.896552,2.115385,4.666667,1.214286,0.875000,2.142857,7.0,...,0.333333,4.000000,0.333333,4.0,0.552326,2.810526,2.244920,1.445450,0.447278,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8743,1.0,2.0,1.0,1.430769,1.698925,8.500000,1.117647,1.545455,1.647059,3.4,...,0.500000,3.000000,0.500000,3.0,2.062176,1.484925,0.594458,2.682204,0.650281,0
8744,1.0,2.0,1.0,0.977011,2.023529,3.000000,1.333333,0.937500,2.066667,7.5,...,0.333333,4.000000,0.333333,4.0,0.595930,2.678049,3.466227,1.288498,0.383086,0
8745,1.0,2.0,1.0,0.900000,2.111111,0.666667,2.500000,0.666667,2.500000,0.8,...,0.857143,2.166667,0.142857,8.0,0.601852,2.661538,1.414914,1.706757,0.168813,0
8746,5.0,1.2,1.0,0.574713,2.740000,1.200000,1.833333,0.375000,3.666667,3.0,...,0.333333,4.000000,0.333333,4.0,0.627907,2.592593,1.723229,1.580306,0.336084,0


In [24]:
stat_df = pd.DataFrame({'n_features':nfeats, 'precision':precs, 'recall':recalls})
stat_df.to_csv('pruning_stats.csv', index=None)

In [25]:
stat_df

,n_features,precision,recall
0,332,0.954448,0.978353
1,249,0.950731,0.968471
2,186,0.955557,0.966392
3,139,0.945260,0.974314
4,104,0.945341,0.970353
5,78,0.946129,0.966431
6,58,0.947445,0.984196
7,43,0.962589,0.964471
8,32,0.950110,0.970392
9,24,0.947062,0.972314
